# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 25 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.12596


extracting tarball to tmp_2212.12596...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.12876


extracting tarball to tmp_2212.12876...

...

extracting tarball to tmp_2212.13146...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.13148


extracting tarball to tmp_2212.13148... done.
Retrieving document from  https://arxiv.org/e-print/2212.13153


extracting tarball to tmp_2212.13153...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.12596-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.12596) | **A Single Binary May Host Recurrent Thermonuclear Supernovae**  |
|| <mark>Kaela J. Lee</mark>, Ken J. Shen |
|*Appeared on*| *2022-12-27*|
|*Comments*| *11 pages, 8 figures*|
|**Abstract**| The most commonly accepted progenitor system for Type Iax supernovae (SNe Iax) is the partial deflagration of a near-Chandrasekhar-mass white dwarf (WD) accreting from a non-degenerate helium donor star, leaving a bound remnant following the explosion. In this paper, we investigate whether the WD remant can undergo multiple SNe during the system's lifetime. We use Modules for Experiments in Astrophysics (MESA) to evolve various single-degenerate binaries to determine which could plausibly undergo multiple SNe Iax due to multiple helium accretion phases. We also investigate the possibility for a subsequent Type Ia SN after the formation of a double WD system. Our work concludes that close binaries with relatively high-mass donors produce the highest probability for several thermonuclear SNe. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.12876-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.12876) | **The Large Sky Area Multi-object Fiber Spectroscopic Telescope (LAMOST)  Quasar Survey: Quasar Properties from Data Release Six to Nine**  |
|| Jun-Jie Jin, et al. -- incl., <mark>Yan-Li Ai</mark>, <mark>Zi-Qi He</mark>, <mark>Yan-xia Zhang</mark> |
|*Appeared on*| *2022-12-27*|
|*Comments*| *arXiv admin note: text overlap with arXiv:1811.01570*|
|**Abstract**| We report the fourth installment in the series of the Large Sky Area Multi-Object Fiber Spectroscopic Telescope (LAMOST) Quasar Survey, which includes quasars observed between September, 2017 and June, 2021. There are in total 13,066 quasars reliably identified, of which 6,685 are newly discovered that are not reported in the SDSS DR14 quasar catalog or Million Quasars catalog. Because LAMOST does not provide accurate absolute flux calibration, we re-calibrate the spectra with the SDSS/Pan-STARRS1 multi-band photometric data. The emission line properties of H$\alpha$, H$\beta$, Mg\,{\sc ii} and C\,{\sc iv}, and the continuum luminosities are measured by fitting the re-calibrated spectra. We also estimate the single-epoch virial black hole masses ($\rm M_{BH}$) using the derived emission line and continuum parameters. This is the first time that the emission line and continuum fluxes were estimated based on LAMOST re-calibrated quasar spectra. The catalog and spectra for these quasars are available online. After the nine-year LAMOST quasar survey, there are in total 56,175 identified quasars, of which 24,127 are newly discovered. The LAMOST quasar survey not only discovers a great number of new quasars, but also provides a database for investigating the spectral variability of the quasars observed by both LAMOST and SDSS, and finding rare quasars including changing-look quasars and broad absorption line quasars. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13058-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13058) | **Jets in a Gamma-Ray Burst During its Prompt Emission: Evolution of  Lorentz Factor**  |
|| <mark>Jing Li</mark>, et al. -- incl., <mark>Da-Bin Lin</mark>, <mark>Wen-Qiang Liang</mark>, <mark>En-Wei Liang</mark> |
|*Appeared on*| *2022-12-27*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**| Knowledge about the Lorentz factor and its evolution of relativistic jets in gamma-ray bursts (GRBs) is crucial to understand their physics. An exact value of bulk Lorentz factor can be estimated based on a high-energy spectral cutoff, which may appear in GRBs' prompt emission owing to the absorption of photon-photon pair production. In this work, we focus on the investigation of the bulk Lorentz factor evolution of jets in an individual burst. Based on \textsl{Fermi} observations, we search for the bursts with multiple $\gamma$-ray pulses characterized by a high-energy spectral cutoff, and nine GRBs are obtained. Together with the estimation of the pulse duration and radiation spectrum, the Lorentz factor of jets corresponding to different pulses in an individual GRB are estimated. It is shown that the Lorentz factor of jets in an individual GRB fluctuates within a certain range and without a general trend in these nine GRBs. In addition, the Lorentz factors of the jets in GRBs~130821A, 160509A and 160625B seem to increase with time. We also study the relations among $L_{\rm iso }$, $E_{\rm p,z}$, and $\Gamma$ for the pulses in our sample, which is found to be consistent with that found in previous works. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13119-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13119) | **Accretion geometry of the neutron star low mass X-ray binary Cyg X-2  from X-ray polarization measurements**  |
|| R. Farinelli, et al. -- incl., <mark>J. Heyl</mark>, <mark>I. Liodakis</mark> |
|*Appeared on*| *2022-12-27*|
|*Comments*| *10 pages, 7 figures, accepted for publication in MNRAS*|
|**Abstract**| We report spectro-polarimetric results of an observational campaign of the bright neutron star low-mass X-ray binary Cyg X-2 simultaneously observed by IXPE, NICER and INTEGRAL. Consistently with previous results, the broad-band spectrum is characterized by a lower-energy component, attributed to the accretion disc with $kT_{\rm in} \approx$ 1 keV, plus unsaturated Comptonization in thermal plasma with temperature $kT_{\rm e} = 3$ keV and optical depth $\tau \approx 4$, assuming a slab geometry. We measure the polarization degree in the 2-8 keV band $P=1.8 \pm 0.3$ per cent and polarization angle $\phi = 140^{\circ} \pm 4^{\circ}$, consistent with the previous X-ray polarimetric measurements by OSO-8 as well as with the direction of the radio jet which was earlier observed from the source. While polarization of the disc spectral component is poorly constrained with the IXPE data, the Comptonized emission has a polarization degree $P =4.0 \pm 0.7$ per cent and a polarization angle aligned with the radio jet. Our results strongly favour a spreading layer at the neutron star surface as the main source of the polarization signal. However, we cannot exclude a significant contribution from reflection off the accretion disc, as indicated by the presence of the iron fluorescence line. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13146-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13146) | **Dark energy and matter interacting scenario can relieve $H_0$ and $S_8$  tensions**  |
|| <mark>Li-Yang Gao</mark>, She-Sheng Xue, <mark>Xin Zhang</mark> |
|*Appeared on*| *2022-12-27*|
|*Comments*| *7 pages, 4 figures*|
|**Abstract**| In this work, we consider a new cosmological model (named $\tilde\Lambda$CDM) in which the vacuum energy interacts with matter and radiation, and test this model using the current cosmological observations. We find that this model can significantly relieve the $H_0$ tension, and at the same time it can also slightly reduce the $S_8$ tension, which cannot be easily observed in other cosmological models. Using the CMB+BAO+SN (CBS) data to constrain the model, we obtain the results of $H_0=70.6^{+1.4}_{-1.7}~\rm{km~s^{-1} Mpc^{-1}}$ and $S_8=0.820\pm 0.011$, and thus the $H_0$ and $S_8$ tensions are relieved to $1.28\sigma$ and $2.67\sigma$, respectively. However, in this case the $\tilde\Lambda$CDM model is not favored by the data, compared with $\Lambda$CDM. We find that when the $H_0$ and $S_8$ data are added into the data combination, the situation is significantly improved. In the CBS+$H_0$ case, we obtain the result of $H_0=72.2\pm 1.2$ ${\rm km~s^{-1}~Mpc^{-1}}$, which relieves the $H_0$ tension to $0.53\sigma$, and in this case the model is favored over $\Lambda$CDM. In the CBS+$H_0$+$S_8$ case, we get a synthetically best situation, $H_0=71.9\pm 1.1$ ${\rm km~s^{-1}~Mpc^{-1}}$ and $S_8=0.8071\pm 0.0099$, in which the $H_0$ and $S_8$ tensions are relived to $0.75\sigma$ and $2.09\sigma$, respectively. In this case, the model is most favored by the data. Therefore, such a cosmological model can greatly relieve the $H_0$ tension, and at the same time it can also effectively alleviate the $S_8$ tension. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13148-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13148) | **Distribution of fast radio burst dispersion measures in CHIME/FRB  Catalog 1: implications on the origin of FRBs**  |
|| <mark>Jianwei Zhang</mark>, et al. -- incl., <mark>Chengmin Zhang</mark>, <mark>Di Li</mark>, <mark>Xianfei Zhang</mark> |
|*Appeared on*| *2022-12-27*|
|*Comments*| *7 pages, 2 figures, accepted for publication in Astronomy Reports*|
|**Abstract**| Recently, CHIME/FRB project published its first fast radio burst (FRB) catalog (hereafter, Catalog 1), which totally contains 536 unique bursts. With the help of the latest set of FRBs in this large-size catalog, we aim to investigate the dispersion measure (DM) or redshift ($z$) distribution of the FRB population, and solution of this problem could be used to clarify the question of FRB origin. In this study, we adopted the M\&E 2018 model, to fit the observed $z$ distribution of FRBs in Catalog 1. In the M\&E 2018 model, we are mostly interested in the $\Phi(z)$ function, i.e., number of bursts per proper time per comoving volume, which is represented by the star formation rate (SFR) with a power-law index $n$. Our estimated value of $n$ is $0.0_{-0.0}^{+0.6}$ ($0.0_{-0.0}^{+2.1}$) at the 68 (95) per cent confidence level, implying that the FRB population evolves with redshift consistent with, or faster than, the SFR. Specially, the consistency of the $n$ values estimated by this study and the SFR provides a potential support for the hypothesis of FRBs originating from young magnetars. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13153-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13153) | **The Statistical Polarization Properties of Coherent Curvature Radiation  by Bunches: Application to Fast Radio Burst Repeaters**  |
|| <mark>Ze-Nan Liu</mark>, Wei-Yang Wang, Yuan-Pei Yang, Zi-Gao Dai |
|*Appeared on*| *2022-12-27*|
|*Comments*| *12 pages, 13 figures, accepted for publication in ApJ*|
|**Abstract**| Fast radio bursts (FRBs) are extragalactic radio transients with millisecond duration and extremely high brightness temperature. Very recently, some highly circularly polarized bursts were found in a repeater, FRB 20201124A. The significant circular polarization might be produced by coherent curvature radiation by bunches with the line of sight (LOS) deviating from the bunch central trajectories. In this work, we carry out simulations to study the statistical properties of burst polarization within the framework of coherent curvature radiation by charged bunches in the neutron star magnetosphere for repeating FRBs. The flux is almost constant within the opening angle of the bunch. However, when the LOS derivates from the bunch opening angle, the larger the derivation, the larger the circular polarization but the lower the flux. We investigate the statistical distribution of circular polarization and flux of radio bursts from an FRB repeater, and find that most of the bursts with high circular polarization have a relatively low flux. Besides, we find that most of the depolarization degrees of bursts have a small variation in a wide frequency band. Furthermore, we simulate the polarization angle (PA) evolution and find that most bursts show a flat PA evolution within the burst phases, and some bursts present a swing of PA. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.13019-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.13019) | **OGLE-2018-BLG-0584 and KMT-2018-BLG-2119: two microlensing events with  two lens masses and two source stars**  |
|| Cheongho Han, et al. -- incl., <mark>Andrew Gould</mark>, <mark>Chung-Uk Lee</mark>, <mark>Seung-Lee Kim</mark>, <mark>Dong-Joo Lee</mark>, <mark>Yongseok Lee</mark> |
|*Appeared on*| *2022-12-27*|
|*Comments*| *9 pages, 9 figures*|
|**Abstract**| We conduct a systematic investigation of the microlensing data collected during the previous observation seasons for the purpose of reanalyzing anomalous lensing events with no suggested plausible models. We find that two anomalous lensing events OGLE-2018-BLG-0584 and KMT-2018-BLG-2119 cannot be explained with the usual models based on either a binary-lens single-source (2L1S) or a single-lens binary-source (1L2S) interpretation. We test the feasibility of explaining the light curves with more sophisticated models by adding an extra lens (3L1S model) or a source (2L2S model) component to the 2L1S lens-system configuration. We find that a 2L2S interpretation well explains the light curves of both events, for each of which there are a pair of solutions resulting from the close and wide degeneracy. For the event OGLE-2018-BLG-0584, the source is a binary composed of two K-type stars, and the lens is a binary composed of two M dwarfs. For KMT-2018-BLG-2119, the source is a binary composed of two dwarfs of G and K spectral types, and the lens is a binary composed of a low-mass M dwarf and a brown dwarf. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

148  publications in the last 7 days.
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers